In [ ]:
import fiftyone as fo
import fiftyone.zoo as foz
dataset = foz.load_zoo_dataset(
    "coco-2017",
    split="train",
    max_samples=100,
    shuffle=True,
    include_id=True,
    seed=5212
)
id_and_pathfile=([(sample['coco_id'],sample['filepath']) for sample in dataset.to_dict()['samples']])

 100% |██████|    1.9Gb/1.9Gb [2.5s elapsed, 0s remaining, 763.1Mb/s]      
Extracting annotations to '/root/fiftyone/coco-2017/raw/instances_train2017.json'
 100% |██████████████████| 100/100 [2.5s elapsed, 0s remaining, 40.4 images/s]      
Writing annotations for 100 downloaded samples to '/root/fiftyone/coco-2017/train/labels.json'
Dataset info written to '/root/fiftyone/coco-2017/info.json'
Loading 'coco-2017' split 'train'
 100% |█████████████████| 100/100 [537.3ms elapsed, 0s remaining, 186.1 samples/s]      
Dataset 'coco-2017-train-100' created
 100% |█████████████████| 100/100 [373.4ms elapsed, 0s remaining, 267.8 samples/s]      


In [ ]:
!pip install -q transformers flax==0.3.5
!pip install -q git+https://github.com/patil-suraj/vqgan-jax.git  # VQGAN model in JAX
!pip install -q git+https://github.com/borisdayma/dalle-mini.git  # Model files

You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [ ]:
from dalle_mini.model import CustomFlaxBartForConditionalGeneration
from transformers import BartTokenizer
import jax
import random
from tqdm.notebook import tqdm, trange
from vqgan_jax.modeling_flax_vqgan import VQModel
import numpy as np
from PIL import Image
from transformers import CLIPProcessor, FlaxCLIPModel


str_coco_id="""212688;	a close up of a polar bear in a body of water
393274;	a yellow and white train traveling past a train station
427123;	three people in formal military uniforms, two are wearing medals around their necks
405324;	the girl has a plate with a sandwich on it
300357;	a woman in a skirt and colorful shirt kneeling down on floral fabric
57340;	there is a tower that has a american flag at the top and a clock on it
198654;	there are people playing a game of baseball
518361;	three people standing at the zoo watching a elephant
525640;	a banana dressed up to look like a person infront of a paper
526347;	a skateboarder on a skateboard in a skate park
4130;	a bus parked on a street in front of several buildings
522661;	two people holding suitcases walking towards a greyhound bus
310526;	a traffic light with a green signal next to tall buildings
497591;	a small child is sitting on a bench outside
338991;	a man is on a tennis court playing tennis
446463;	an old fashioned photograph of a fleet of buses
50356;	a lot of flowers that are sitting on a table
301402;	a man sits at a table with a laptop computer on his knees
56624;	a wood bench is outside covered in snow
21097;	the meal on the table is ready to be eaten
14824;	a group of people dressed as zombies near a stop sign
317730;	a man is holding a surfboard decorated with the image of a woman
160703;	two pizza's on sliver pans on a table with utensils
302313;	the thin cow eats some grass next to the tree
104842;	a skateboarded takes to the air in a skateboard park
569865;	autumn trees are next to hanging stop lights.
62824;	a street sign stands on the 8500 west block of hollywood boulevard.
327802;	a street sign set in the edge of the sidewalk that says stockton st.
385341;	a couple of people that are sitting down with wii remotes
370121;	a bench sits in a field by the water while it is cloudy.
475277;	a white horse looking up for a photo at a fence side.
140122;	a computer monitor, keyboard, and tower with peripherals and plugs sit on a desk.
119502;	toddler boy in a tie playing in a dish of mud.
413868;	a man in the woods throws a frisbee
12650;	several people gathered outside of a bar at night.
269338;	a baseball player sliding into a base with another player bent over with glove extended toward a flying baseball.
514601;	the train platform has green benches on it.
495456;	a person using an oven mitt to take something from the oven.
106513;	bus at rainy bus stop with commuters waiting
380258;	a pigeon sitting outside a window of a high rise building
61354;	an open market selling assorted fruits with bananas hanging
506034;	there is a side view mirror and traffic lights in this picture
239593;	a black and white picture of a train going down a track with a city in the background
103375;	a bed has its covers pulled back in a dark room.
452687;	a person's foot strapped into a black snowboard.
430885;	a cloudy skyline with silhouettes of industrial buildings.
292152;	a toasted ham sandwich with a pickle on a white plate.
90024;	a giant teddy bear in front of some trees at the mall
575331;	an octopus kite is being flown next to a scuba diver kite above a city park.
404917;	a baseball player throwing a baseball on a field.
13176;	a man bending to look into a small fridge
230227;	a cat sitting on a table with a blurred image in the background
122355;	four people smiling with tennis rackets in their hands
359143;	a blurry photo of a cyclist on a forest path
135288;	a young man holding a plate and eating a piece of cake
273029;	a piece of pizza with many topping on it
185633;	an old bathroom with peeling paint on the walls and lots of toiletries scattered around
270222;	several small clocks are all connected with human-like ornaments
439868;	a skateboarder is jumping onto a rail near crowd
424174;	a tray heaped with fruit including banans apples oranges and limes
344720;	several cars at an intersection on a city stree
292782;	a group of people flying their kites on a sunny day
102765;	two young girls sitting on their luggage outside of a brick building
484149;	a field of yellow grass with sheep grazing and standing throughout the areas
27593;	a man flying through the air on top of a skateboard
266165;	a cat standing in one food dish while eating out of another dish
125070;	a blue clock pole outside an older building
78771;	a cat sitting next to a laptop computer on a table
291606;	a woman standing outside of a bus with asian characters written on it
423116;	on a kitchen counter are a laptop, candy, and cooking supplies
405762;	a man is running while carrying a football
41001;	two adult elephants and one baby elephant walking in a grassy area
201925;	something baking in a pan in the oven and rising out of the pan
380734;	several diners eating at a riverside outdoor restaurant
487462;	a boy is preparing to toss a frisbie while another boy is sitting in the background in a park
455776;	a young boy is preparing to catch a baseball.
440895;	a white horse can be seen in the window of a brick building.
55627;	a bathroom with a mirror above the sink, a towel rack and toilet.
436111;	a white bathroom has dirt all over the floor.
492286;	a big train is about to leave and go away.
149892;	a bench and chair are seen on the bank of a river.
493196;	a man with helmet skating past some bicycle riders on the road
182575;	this is a plate of broccoli and cauliflower
203559;	a man is inspecting a baby elephants eye
519950;	a man is in mid air over a tennis court.
27276;	a dog is under the bed eating a box
108425;	a man tossing a neon green frisbee in front of a house
158713;	a man who is driving a motorcycle pulling a cart with two people.
5701;	a person standing on a small paved square and holding a yellow frisbee
254994;	a man holding a drinking glass and smiling.
256799;	a chandelier sitting above a spiral stair case.
524981;	a dog sitting on a boat behind the sail.
509839;	a public bathroom that is empty and not in use.
194120;	a cat sitting on a windowsill next to a painted pumpkin.
197329;	some zebras and a giraffe tall brown grass and trees
432637;	a group of people riding a carousel with trees in the background.
298014;	a large architectural structure near a park
250225;	a display of fruit print dishes in a store
484457;	a large stack of square shaped cakes sitting on a lit table.
541039;	a group of people holding umbrellas standing behind a sign for a umbrella drive.
423313;	a group of motorcycles parked in front of a building.
"""

# make sure we use compatible versions
DALLE_REPO = "flax-community/dalle-mini"
DALLE_COMMIT_ID = "4d34126d0df8bc4a692ae933e3b902a1fa8b6114"
# make sure we use compatible versions
VQGAN_REPO = "flax-community/vqgan_f16_16384"
VQGAN_COMMIT_ID = "90cc46addd2dd8f5be21586a9a23e1b95aa506a9"
# set up model and processor
clip = FlaxCLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# set up tokenizer and model
tokenizer = BartTokenizer.from_pretrained(DALLE_REPO, revision=DALLE_COMMIT_ID)
model = CustomFlaxBartForConditionalGeneration.from_pretrained(
    DALLE_REPO, revision=DALLE_COMMIT_ID
)
vqgan = VQModel.from_pretrained(VQGAN_REPO, revision=VQGAN_COMMIT_ID)

for _,line in enumerate(str_coco_id):
    id_coco, caption= str_coco_id.split(";"))
    # set a prompt
    prompt = caption.strip()

    # tokenize the prompt
    tokenized_prompt = tokenizer(
        prompt, return_tensors="jax", padding="max_length", truncation=True, max_length=128
    )

    n_predictions = 10

    # create random keys
    seed = random.randint(0, 2 ** 32 - 1)
    key = jax.random.PRNGKey(seed)
    subkeys = jax.random.split(key, num=n_predictions)


    encoded_images = [
        model.generate(**tokenized_prompt, do_sample=True, num_beams=1, prng_key=subkey)
        for subkey in tqdm(subkeys)
    ]

    encoded_images = [img.sequences[..., 1:] for img in encoded_images]

    decoded_images = [
        vqgan.decode_code(encoded_image) for encoded_image in tqdm(encoded_images)
    ]

    clipped_images = [img.squeeze().clip(0.0, 1.0) for img in decoded_images]


    # convert to image
    images = [
        Image.fromarray(np.asarray(img * 255, dtype=np.uint8)) for img in clipped_images
    ]

    # evaluate scores
    inputs = processor(text=prompt, images=images, return_tensors="np")
    logits = clip(**inputs).logits_per_image
    scores = jax.nn.softmax(logits, axis=0).squeeze()  # normalize and sum all scores to 1

    # rank images by score
    print(f"Prompt: {prompt}\n")
    for idx in scores.argsort()[::-1]:
        print(f"Score: {scores[idx]}")
        display(images[idx])
        print()

In [ ]:
!git clone https://github.com/cthiounn/images-generated-by-dalle-mini.git

Cloning into 'images-generated-by-dalle-mini'...
remote: Enumerating objects: 6208, done.
remote: Counting objects: 100% (5163/5163), done.
remote: Compressing objects: 100% (5120/5120), done.
remote: Total 6208 (delta 110), reused 5094 (delta 43), pack-reused 1045
Receiving objects: 100% (6208/6208), 77.04 MiB | 22.40 MiB/s, done.
Resolving deltas: 100% (129/129), done.
Checking out files: 100% (6017/6017), done.


In [ ]:
import pandas as pd
df=pd.read_csv("/work/images-generated-by-dalle-mini/minidalle_flickr30k_clip.csv")
df

,1,0,2,flickr
0,48_1206_2.jpeg,25.037296,1206,0
1,995_30781_4.jpeg,23.693491,30781,0
2,186_5947_1.jpeg,28.058033,5947,0
3,211_6661_5.jpeg,27.386684,6661,0
4,544_17045_2.jpeg,25.031023,17045,0
...,...,...,...,...
5989,429_13543_1.jpeg,24.167414,13543,0
5990,960_29489_1.jpeg,26.647877,29489,0
5991,829_25516_4.jpeg,22.392540,25516,0
5992,836_25753_2.jpeg,22.078007,25753,0


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=4f3692ed-5f27-49a4-899a-82a03e72232c' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>